# !python setup.py build_ext --inplace

In [1]:
cd git_retina

/Users/kumarshubham/Desktop/COP/git_retina


In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [3]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = 'resnet50_coco_best_v2.1.0.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

/anaconda2/lib/python2.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

In [5]:
cd ..

/Users/kumarshubham/Desktop/COP


In [6]:
def Visualize(draw,boxes,scores,labels,index=0):
    # visualize detections
    draw=draw[index]
    print(boxes.shape)
    for box, score, label in zip(boxes[index], scores[index], labels[index]):
        # scores are sorted so we can break
        if score < 0.5:
            break
        print(box.shape)
        color = label_color(label)
    
        b = box.astype(int)
        print(b.shape)
        draw_box(draw, b, color=color)
    
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [7]:
import pandas as pd
Annotations=pd.DataFrame([],columns=["path","x1","y1","x2","y2","class_name"])
ClassMap=pd.DataFrame([],columns=["class_name","id"])

In [8]:
CLASSES1=[x for x in os.listdir('Images') if x!='.DS_Store']
ImagePaths=[]
out_classes=[]
for i,class1 in enumerate(CLASSES1):
    ClassMap.loc[i]=[class1,i]
    Folder=os.path.join('Images',class1)
    ImagePaths= ImagePaths+[os.path.join(Folder,x) for x in os.listdir(Folder) if x!='.DS_Store']
    out_classes=out_classes+[class1 for x in os.listdir(Folder) if x!='.DS_Store']
print(len(CLASSES1),len(ImagePaths))
Images = [cv2.imread(image) for image in ImagePaths ]
out_classes=[x.replace for x,y in zip(out_classes,Images) if y is not None]
Images=[image for image in Images if image is not None]

Sizes=[image.shape for image in Images ]
Num_Images=len(ImagePaths)
images=[cv2.resize(image,(300,300)) for image in Images]
images=np.stack(images,axis=0)
print(images.shape)

(43, 9938)
(9923, 300, 300, 3)


In [9]:
len(out_classes)

9923

In [10]:
import shutil
shutil.rmtree("Detected")
shutil.rmtree("Not_Detected")
os.makedirs("Detected")
os.makedirs("Not_Detected")
j=0
for class2 in CLASSES1:
    os.makedirs("Detected/"+class2)
    os.makedirs("Not_Detected/"+class2)
Num_Images=len(ImagePaths)
for image_indices in np.array_split(range(Num_Images),(Num_Images/50)+1):   
    # process image
    start = time.time()
    print(image_indices[0],image_indices[-1])
    boxes, scores, labels =  model.predict_on_batch(images[image_indices])
    print("processing time: ", time.time() - start)
    # loop over the detections
    for index in image_indices:
        draw=Images[index]
        Detected=False
        for box, confidence, label in zip(boxes[index-image_indices[0]], scores[index-image_indices[0]], labels[index-image_indices[0]]):
            # scores are sorted so we can break
            if confidence < 0.7:
                break   
            if label==14:
                box=box*[float(Sizes[index][1])/300,float(Sizes[index][0])/300,float(Sizes[index][1])/300,float(Sizes[index][0])/300]
                b = box.astype(int)
                color = label_color(label)
                draw_box(draw, b, color=color)

                caption = "{} {:.3f}".format(out_classes[index], confidence)
                draw_caption(draw, b, caption)
                Detected=True
                Annotations.loc[j]=[ImagePaths[index],box[0],box[1],box[2],box[3],out_classes[index]]
                j+=1
        #if Detected:
            #cv2.imwrite('Detected/'+ImagePaths[index][7:], draw)
        #else:
            #cv2.imwrite('Not_Detected/'+ImagePaths[index][7:], draw)
    print("total time: ", time.time() - start)

(0, 49)
('processing time: ', 56.41153812408447)
('total time: ', 57.13464903831482)
(50, 99)
('processing time: ', 59.39632201194763)
('total time: ', 59.8161940574646)
(100, 149)
('processing time: ', 48.52785801887512)
('total time: ', 48.822808027267456)
(150, 199)
('processing time: ', 47.20721197128296)
('total time: ', 47.65967798233032)
(200, 249)
('processing time: ', 46.972635984420776)
('total time: ', 47.42124080657959)
(250, 299)
('processing time: ', 46.59007406234741)
('total time: ', 47.042267084121704)
(300, 349)
('processing time: ', 47.1353018283844)
('total time: ', 47.84509086608887)
(350, 399)
('processing time: ', 46.82799005508423)
('total time: ', 47.38913917541504)
(400, 449)
('processing time: ', 50.66594910621643)
('total time: ', 51.19585394859314)
(450, 499)
('processing time: ', 77.07547092437744)
('total time: ', 77.92448997497559)
(500, 549)
('processing time: ', 103.26362609863281)
('total time: ', 104.05622816085815)
(550, 599)
('processing time: ', 9

('total time: ', 49.90241885185242)
(4600, 4649)
('processing time: ', 48.63474202156067)
('total time: ', 49.205621004104614)
(4650, 4699)
('processing time: ', 46.66789197921753)
('total time: ', 47.333250999450684)
(4700, 4749)
('processing time: ', 49.18884205818176)
('total time: ', 49.86195087432861)
(4750, 4799)
('processing time: ', 51.302785873413086)
('total time: ', 52.06319880485535)
(4800, 4849)
('processing time: ', 47.750612020492554)
('total time: ', 48.48376798629761)
(4850, 4899)
('processing time: ', 46.52975416183472)
('total time: ', 47.318915128707886)
(4900, 4949)
('processing time: ', 46.03089904785156)
('total time: ', 46.72815704345703)
(4950, 4999)
('processing time: ', 45.77738380432129)
('total time: ', 46.54939293861389)
(5000, 5049)
('processing time: ', 45.756534814834595)
('total time: ', 46.44940686225891)
(5050, 5099)
('processing time: ', 45.8396680355072)
('total time: ', 46.69324016571045)
(5100, 5149)
('processing time: ', 46.59081506729126)
('tot

('processing time: ', 57.29757809638977)
('total time: ', 57.94765090942383)
(9150, 9199)
('processing time: ', 55.799293994903564)
('total time: ', 56.45102500915527)
(9200, 9249)
('processing time: ', 58.01492500305176)
('total time: ', 58.61000895500183)
(9250, 9299)
('processing time: ', 59.961414098739624)
('total time: ', 60.567198038101196)
(9300, 9349)
('processing time: ', 57.53902006149292)
('total time: ', 58.116081953048706)
(9350, 9398)
('processing time: ', 53.904958963394165)
('total time: ', 54.529786109924316)
(9399, 9447)
('processing time: ', 55.36067795753479)
('total time: ', 55.882241010665894)
(9448, 9496)
('processing time: ', 56.15170693397522)
('total time: ', 56.70636296272278)
(9497, 9545)
('processing time: ', 46.52604699134827)
('total time: ', 47.17201209068298)
(9546, 9594)
('processing time: ', 56.26532697677612)
('total time: ', 56.896880865097046)
(9595, 9643)
('processing time: ', 56.55547380447388)
('total time: ', 57.23225498199463)
(9644, 9692)
('

IndexError: index 9923 is out of bounds for axis 0 with size 9923

In [ ]:
# process image
start = time.time()
boxes, scores, labels =  model.predict_on_batch(images[:5])
print("processing time: ", time.time() - start)

In [ ]:
Visualize(Images,boxes*[2.1666666666666665,2, 2.1666666666666665,2],scores,labels,3)

In [ ]:
'Detected/'+ImagePaths[1][7:]

In [ ]:
labels[67]

In [ ]:
print min(Sizes)

In [ ]:
Annotations['x1']=Annotations['x1'].astype(int)
Annotations['x2']=Annotations['x2'].astype(int)
Annotations['y1']=Annotations['y1'].astype(int)
Annotations['y2']=Annotations['y2'].astype(int)

In [ ]:
Annotations.to_csv("Annotations.csv",index=False,header=False)

In [ ]:
ClassMap.to_csv("ClassMap.csv",index=False,header=False)

In [ ]:
Annotations